In [1]:
import pandas as pd
from path import Path, getcwdu
import glob
import os
from pathlib import PurePath
import copy
import random
import shutil

In [2]:
#full_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob('../employee_filings/*.gz')]

In [3]:
random.seed(14)

In [ ]:
#test_path_list = random.sample(full_path_list, int(len(full_path_list)*.2))
#test_path =PurePath(os.getcwd()).joinpath('../employee_filings_test/')

In [59]:
subset_path =PurePath(os.getcwd()).joinpath('subset_filings/')

In [6]:
#for file in test_path_list:
#    test_dest = test_path.joinpath(PurePath(file).name)
#    shutil.move(file, test_dest)

In [9]:
full_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob('../employee_filings/*.gz')]
full_file_list = [PurePath(file).name for file in glob.iglob('../employee_filings/*.gz')]
full_accession_ids = [PurePath(file).stem.replace('.html', '') for file in full_file_list]
full_cik_nbrs = [x.split(sep='-')[0] for x in full_accession_ids]

In [10]:
#val_accession_ids = random.sample(full_accession_ids, int(len(full_accession_ids)*.25))

In [36]:
train_accession_ids = [x for x in full_accession_ids if x not in val_accession_ids]
test_accession_ids = [PurePath(file).stem.replace('.html', '') for file in glob.iglob('../employee_filings_test/*.gz')]

Create lists of accession IDs for easy reuse (currently being treated as proprietary, since it would take some work to compile the master list)

In [55]:
pd.DataFrame(val_accession_ids).to_csv('../val_accession_ids.csv', index=False)
pd.DataFrame(train_accession_ids).to_csv('../train_accession_ids.csv', header=False, index=False)
pd.DataFrame(test_accession_ids).to_csv('../test_accession_ids.csv', index=False)

In [52]:
print('Train IDs: ' + str(len(train_accession_ids)))
print('Validation IDs: ' + str(len(val_accession_ids)))
print('Test IDs: ' + str(len(test_accession_ids)))

Train IDs: 1667
Validation IDs: 555
Test IDs: 555


In [16]:
full_labeled_df = pd.read_excel('../data/employee_count_paragraphs.xlsx')

Label rows by split (test, val, train)

In [44]:
full_labeled_df['split'] = 'train'
full_labeled_df.loc[full_labeled_df.accession_number.isin(test_accession_ids), 'split'] = 'test'
full_labeled_df.loc[full_labeled_df.accession_number.isin(val_accession_ids), 'split'] = 'val'

Write to excel file with split labels

In [46]:
full_writer = pd.ExcelWriter('../data/full_employee_count_paragraphs.xlsx')
full_labeled_df.to_excel(full_writer, index=False)
full_writer.save()

In [ ]:
test_labeled_df = full_labeled_df[full_labeled_df.accession_number.isin(test_accession_ids)]
train_val_labeled_df = full_labeled_df[full_labeled_df.accession_number.isin(train_accession_ids + val_accession_ids)]

subset_df = train_val_labeled_df[train_val_labeled_df['split'] == 'train'].head(20)

In [58]:
subset_file_list = [file for file in full_file_list if PurePath(file).stem.replace('.html', '') in subset_df.accession_number.unique().tolist()]

In [61]:
for file in subset_file_list:
    file_orig = PurePath(os.getcwd()).joinpath('../employee_filings/').joinpath(file)
    file_dest = subset_path.joinpath(PurePath(file).name)
    shutil.copy(file_orig, file_dest)

In [48]:
print('Train-Validation DF rows: ' + str(train_val_labeled_df.shape[0]))
print('Test DF rows: ' + str(test_labeled_df.shape[0]))

Train-Validation DF rows: 2625
Test DF rows: 653


Write test rows to separate file

In [49]:
test_writer = pd.ExcelWriter('../data/test_employee_count_paragraphs.xlsx')
test_labeled_df.to_excel(test_writer, index=False)
test_writer.save()

Write train + validation rows to separate Excel to help avoid Test Data leakage

In [50]:
train_val_writer = pd.ExcelWriter('../data/train_val_employee_count_paragraphs.xlsx')
train_val_labeled_df.to_excel(train_val_writer, index=False)
train_val_writer.save()

Write subset df for semi-supervised learning to separate Excel file

In [57]:
subset_writer = pd.ExcelWriter('../data/subset_employee_count_paragraphs.xlsx')
subset_df.to_excel(subset_writer, index=False)
subset_writer.save()